# Lecture 6 - Scikit-learn

Today we're going to cover the popular machine learning library [Scikit-learn](http://scikit-learn.org/stable/)

First:
```bash
conda install scikit-learn
```

Note, when importing scikit-learn, you do
```python
import sklearn # not "scikit-learn"
```

This library can do just about anything you would learn in an introductory machine learning class (although it doesn't really do deep learning).  This includes:

* Regression
* SVMs
* Decision trees/random forests
* dimensionality reduction
* clustering
* validation
* ...

Supervised learning:
* Regression and classification methods
* All types of models: logistic regression, ridge, SVM, lasso regression, decision trees... up to Neural networks (no GPU support)
* Stochastic Gradient Descent, Nearest-Neighbors,
* Also features semi-supervised learning, ensemble methods, feature selection methods, Naive Bayes, and Isotonic Regression

Unsupervised learning:
* Gaussian Mixture Models, Manifold Learning
* Clustering, Bi-clustering
* PCA, LDA, Outlier detection, Covariance estimation

You may wish to check out [some examples](http://scikit-learn.org/stable/auto_examples/)

As usual, this class will assume you have some passing familiarity with at least something above, since this class isn't really trying to tell you *why* you may wish to classify something or do a regression, just *how* to do it in Python (or at least point you in that direction).

## Loading an example dataset

First we will load some data to play with. The data we will use is a very simple
flower database known as the Iris dataset.

We have 150 observations of the iris flower specifying some measurements: 

- sepal length, sepal width, petal length and petal width together with its subtype:
*Iris setosa*, *Iris versicolor*, *Iris virginica*.

Yes, we saw this last class as well.

In [ ]:
import numpy as np
import sklearn

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

This data is stored in the `.data` member, which is a `(n_samples, n_features)`
array.

In [ ]:
end_string = '\n' + '--'*25 + '\n'
print(iris.keys(), end = end_string)
print(iris.target.shape, end = end_string)

The class of each observation is stored in the `.target` attribute of the
dataset. This is an integer 1D array of length `n_samples`:

In [ ]:
print(iris.target.shape)
np.unique(iris.target)

# The Scikit-learn Paradigm

Almost everything you do in scikit learn will be a variation of the same basic pattern, regardless of the specifics of what you're actually doing

1. Load the model class
2. Initialize a model (this is where you specify parameters)
3. Fit your model to data
4. (Context dependent) - predict, visualize, explore your fit model

# Basic Classification

We'll start with a nearest neighbor classifier.

## k-Nearest neighbors classifier

The simplest possible classifier is the nearest neighbor: given a new
observation, take the label of the training samples closest to it in
*n*-dimensional space, where *n* is the number of *features* in each sample.

The k-nearest neighbors classifier internally uses an algorithm based on
ball trees to represent the samples it is trained on.

![image](./img/iris_knn.png)

Note that most functionality in `sklearn` lives in modules, so you'll need to do something like
```python
from sklearn import neighbors
```

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn import neighbors # access to model class

knn = neighbors.KNeighborsClassifier() # initialize a model (default parameters)

knn.fit(iris.data, iris.target) # fit the model

knn.predict([[0.1, 0.2, 0.3, 0.4]]) # do something with the model

In [ ]:
knn.get_params()

## Training set and testing set

When experimenting with learning algorithms, it is important not to test the
prediction of an estimator on the data used to fit the estimator. 

Indeed, with the kNN estimator, we would always get perfect prediction on the training set.

In [ ]:
### Manually
perm = np.random.permutation(iris.target.size)
iris.data = iris.data[perm]
iris.target = iris.target[perm]
knn.fit(iris.data[:100], iris.target[:100])
knn.score(iris.data[100:], iris.target[100:])

In [ ]:
# Preferred
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# split holding out 40 % 
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=0)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# We are drastically reducing the size of our training data, better to do k-fold cross validation 
scores = cross_val_score(knn, iris.data, iris.target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Other Classifiers

SVM, decision trees, random forests...

In [ ]:
from sklearn import svm

model = svm.SVC() # you can pass in various key-word arguments

model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

# Clustering

## K-means

A simple clustering algorithm is k-means. This divides a set into *k*
clusters, assigning each observation to a cluster so as to minimize the distance
of that observation (in *n*-dimensional space) to the cluster's mean; the means
are then recomputed. This operation is run iteratively until the clusters
converge, for a maximum for `max_iter` rounds.

(An alternative implementation of k-means is available in SciPy's `cluster`
package. The `scikit-learn` implementation differs from that by offering an
object API and several additional features, including smart initialization.)

[sklearn kmeans](http://scikit-learn.org/stable/modules/clustering.html#k-means)

In [ ]:
from sklearn import cluster
k_means = cluster.KMeans(n_clusters=3)
labels= k_means.fit_predict(iris.data)
print(labels[::10])
print(iris.target[::10])

## Other Clustering Methods

Most standard clustering algorithms are available in scikit-learn

[clustering in sklearn](http://scikit-learn.org/stable/modules/clustering.html)

In [ ]:
agglom = cluster.AgglomerativeClustering(n_clusters=3, linkage="single")
labels= k_means.fit_predict(iris.data)
print(labels[::10])
print(iris.target[::10])

# Regression

In regression, we're looking to predict a response $y$ from data $X$.  Scikit learn implements most basic regression models, as well as some less standard ones.

If you're familiar with R, the models should be familiar, but the API is new.

[sklearn regression](http://scikit-learn.org/stable/modules/linear_model.html)

## Logistic Regression

Let's do something a little less trivial than what we have above.  We'll use the pandas, patsy, and statsmodels packages

```bash
conda install pandas statsmodels patsy
```

[statsmodels](https://www.statsmodels.org/stable/index.html) is another python library for statistical estimation problems.  We'll use it for an included dataset.

[patsy](https://patsy.readthedocs.io/en/latest/) helps specify statistical models.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import numpy as np

We'll use the following data set, which contains data about the incidence of extramarital affairs in marriages

In [ ]:
print(sm.datasets.fair.SOURCE)
print(sm.datasets.fair.NOTE)

In [ ]:
# load dataset
data = sm.datasets.fair.load_pandas().data

# add "affair" column: 1 represents having affairs, 0 represents not
data['affair'] = (data.affairs > 0).astype(int)
data

In [ ]:
print("Affair proportion by children: \n \n {}\n".format(data.groupby('children')['affair'].mean()))
print("Affair proportion by age: \n \n {}".format(data.groupby('age')['affair'].mean()))

In [ ]:
# groups marriages by how they are rated by the couple
data.groupby('rate_marriage').mean()

We'll visualize a histogram of education levels of the couples

In [ ]:
data.educ.hist()
plt.title('Histogram of Education')
plt.xlabel('Education Level')
plt.ylabel('Frequency');

In [ ]:
data.rate_marriage.hist()
plt.title('Histogram of Marriage Rating')
plt.xlabel('Marriage Rating')
plt.ylabel('Frequency');

In [ ]:
data.corr()

In [ ]:
plt.imshow(data.corr())
plt.colorbar()
plt.show()

In [ ]:
pd.plotting.scatter_matrix(data, figsize=(15, 15))
plt.show()

In [ ]:
affair_yrs_married = pd.crosstab(data.yrs_married, data.affair.astype(bool))
affair_yrs_married.div(affair_yrs_married.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True, figsize=(10,10))
plt.title('Affair Percentage by Years Married')
plt.xlabel('Years Married')
plt.ylim([0,1.25])
_ = plt.ylabel('Percentage')

Now we'll use the [patsy](https://patsy.readthedocs.io/en/latest/) library to create some data matrices from our data frames.

Our features will be all the (non-affair) features in the original dataset, and the response will be the presence of an affair.

In [ ]:
from patsy import dmatrices

In [ ]:
# create dataframes with an intercept column and dummy variables for
# occupation and occupation_husb
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
                  religious + educ + C(occupation) + C(occupation_husb)',
                  data, return_type="dataframe")
print(X.columns)

In [ ]:
X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
                        'C(occupation)[T.3.0]':'occ_3',
                        'C(occupation)[T.4.0]':'occ_4',
                        'C(occupation)[T.5.0]':'occ_5',
                        'C(occupation)[T.6.0]':'occ_6',
                        'C(occupation_husb)[T.2.0]':'occ_husb_2',
                        'C(occupation_husb)[T.3.0]':'occ_husb_3',
                        'C(occupation_husb)[T.4.0]':'occ_husb_4',
                        'C(occupation_husb)[T.5.0]':'occ_husb_5',
                        'C(occupation_husb)[T.6.0]':'occ_husb_6'})

In [ ]:
# flatten y dataframe into a response array in numpy
y = np.ravel(y)

In [ ]:
X

Now we're ready to fit a logistic regression model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

In [ ]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(X, y)

# check the accuracy on the training set
model.score(X, y)

In [ ]:
y.mean()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

In [ ]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print("Predicted {} affairs in {} points".format(predicted.sum(), X_test.shape[0]))

In [ ]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print(probs)

Now we'll create an [ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) to evaluate the model's ability to predict.

In [ ]:
# generate evaluation metrics

roc_auc = metrics.roc_auc_score(y_test, probs[:, 1])
acc = metrics.accuracy_score(y_test, predicted)
print("Accuracy score: {}".format(acc))
print("ROC-AUC score {}".format(roc_auc))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, probs[:, 1], pos_label=1)
plt.figure(figsize=(10,10))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
conf_matrix= metrics.confusion_matrix(y_test, predicted)
print(conf_matrix)

In [ ]:
# Classification report
report =metrics.classification_report(y_test, predicted)
print(report)

## Attribution

Portions of this notebook ar a Jupyter Notebook port of the `scikit-learn` lecture from the
open source [Scipy Lecture Notes][scipy-lec-notes] by Fabian Pedregosa and Gael
Varoquaux.

[scipy-lec-notes]: http://www.scipy-lectures.org/